# "지하철 이용승객 분석" EDA 프로젝트(1)
- 이번 프로젝트에서는 EDA가 무엇이고 , 실습을 통해 EDA 하는 방법을 배워보겠습니다.
- EDA(Exploratory Data Analysis 탐색적 데이터 분석)는 데이터를 이해하는 과정이며 
- 통계적 요약, 분포 파악 및 시각화 등의 기법을 통해 직관적으로 데이터 특성 파악 할수 있습니다.
- 결국, EDA를 통해 데이터에서 트랜드 분석, 인사이트 도출하게 됩니다. 


## 실습 데이터 
- 지하철 승하자 이용객 데이터 : 2019.01.01 ~ 2019.06.30 
  - 서울열린데이터광장 :  http://data.seoul.go.kr/dataList/OA-12914/A/1/datasetView.do
- 지하철 노선 정보 데이터

## EDA 실습을 통해 해결해야 할 질문들
 
- 2019.01~06 중에 언제 지하철을 가장 많이 이용했을까? (기준: 승하자총승객수)
- 1월~6월중에 5월에 지하철 승객수가 많다? (기준 :승하자총승객수)
- 요일중에서 목요일에 지하철 승객수가 많다? (기준 :승하자총승객수)
- 일자별(월일별) 승하차승객수 그래프 그려 볼까요? (기준 :승하자총승객수)
- 가장 승객이 많이 타는 승차역은?
- 노선별로 역별/요일별 승차승객수를 비교해 볼수 있을까? (1~9호선, 역별/요일별 heatmap)
- 1호선에서 가장 하자를 많이 하는 역은? (groupby)
- 2호선중에서 어느 역에서 승차가 가장 많이 발생할까? (Folium 역 표시)

## 해당 Jupyter Notebook에서 실습 내용
- 필요한 라이브러리 가져오기
- 필요한 파일 불러오고 병합하기(read_csv, merge)
- Feature Engineering : 요일, 연월, 월일, 승하자총승객수 컬럼 추가
- 데이터 저장하기

<br>

# 1. 필요한 라이브러리 가져오기

In [27]:
# 필요한 라이브러리를 불러옵니다.

import pandas as pd


# 2 raw 파일 불러오기
- 열차/노선/지하철역별 승하자 고객수
- 19년 상반기(19년 1월 ~ 19년 6월)

In [28]:
# 작업할 여러개의 파일 중 하나의 파일을 불러옵니다.

file ='C:/Users/limjy/Desktop/4학년 1학기/AI+X 선도인재양성/3주차_지하철 이용승객 EDA분석_숙제_20220914/실습/rawfiles/rawfiles/CARD_SUBWAY_MONTH_201901.csv'
pd.read_csv(file)


,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20190101,2호선,을지로4가,3862.0,3728.0,20190104
1,20190101,3호선,을지로3가,8104.0,7554.0,20190104
2,20190101,2호선,을지로입구,22478.0,21330.0,20190104
3,20190101,2호선,시청,8381.0,6049.0,20190104
4,20190101,6호선,동묘앞,8045.0,8504.0,20190104
...,...,...,...,...,...,...
18329,20190131,우이신설경전철,북한산보국문,6195.0,5786.0,20190203
18330,20190131,우이신설경전철,정릉,4600.0,4211.0,20190203
18331,20190131,4호선,성신여대입구,3922.0,4289.0,20190203
18332,20190131,6호선,보문,1659.0,1693.0,20190203


In [29]:
raw = pd.read_csv(file)
raw.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20190101,2호선,을지로4가,3862.0,3728.0,20190104
1,20190101,3호선,을지로3가,8104.0,7554.0,20190104
2,20190101,2호선,을지로입구,22478.0,21330.0,20190104
3,20190101,2호선,시청,8381.0,6049.0,20190104
4,20190101,6호선,동묘앞,8045.0,8504.0,20190104


In [30]:
# info() 이용해 데이터 구조를 살펴볼수 있습니다.

raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18334 entries, 0 to 18333
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   사용일자    18334 non-null  int64  
 1   노선명     18334 non-null  object 
 2   역명      18334 non-null  object 
 3   승차총승객수  18334 non-null  float64
 4   하차총승객수  18334 non-null  float64
 5   등록일자    18334 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 859.5+ KB


## 참고) 판다스로 파일 읽어오기

### 데이터 파일 읽기 : read_excel / read_csv
- pd.read_excel('파일경로+파일명.xlsx')
- pd.read_csv('파일경로+파일명.csv', encoding='utf-8')
- pd.read_csv('파일경로+파일명.csv', encoding='cp949') # MS 엑셀에 저장된 경우

In [31]:
# 이번에 여러 파일을 불러오겠습니다.
# 2개 파일 불러와서 합치기

raw = pd.DataFrame()
raw.head()

file1 = 'C:/Users/limjy/Desktop/4학년 1학기/AI+X 선도인재양성/3주차_지하철 이용승객 EDA분석_숙제_20220914/실습/rawfiles/rawfiles/CARD_SUBWAY_MONTH_201901.csv'
file2 = 'C:/Users/limjy/Desktop/4학년 1학기/AI+X 선도인재양성/3주차_지하철 이용승객 EDA분석_숙제_20220914/실습/rawfiles/rawfiles/CARD_SUBWAY_MONTH_201902.csv'

temp1 = pd.read_csv(file1)
temp2 = pd.read_csv(file2)

# pandas concat 합치때, 인덱스 번호 그래로 간다. 옵션 ignore_index=True 사용하면 인덱스 새로 생성
raw = pd.concat([temp1, temp2], axis=0)


In [32]:
raw

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20190101,2호선,을지로4가,3862.0,3728.0,20190104
1,20190101,3호선,을지로3가,8104.0,7554.0,20190104
2,20190101,2호선,을지로입구,22478.0,21330.0,20190104
3,20190101,2호선,시청,8381.0,6049.0,20190104
4,20190101,6호선,동묘앞,8045.0,8504.0,20190104
...,...,...,...,...,...,...
16538,20190228,2호선,을지로4가,6128.0,7087.0,20190303
16539,20190228,2호선,동대문역사문화공원,3387.0,4061.0,20190303
16540,20190228,5호선,청구,4920.0,4474.0,20190303
16541,20190228,1호선,가산디지털단지,51729.0,52043.0,20190303


---

# Q) 폴더에 있는 모든 파일을 불러와서 병합할수 있을까?

In [33]:
# 폴더/파일을 관리하는 os 라이브러리를 불러옵니다.

import os


In [34]:
# os.listdir()을 이용해 폴더에 있는 파일 리스트를 살펴봅니다.

os.listdir()


['.ipynb_checkpoints',
 '1.지하철 이용승객 분석 EDA 실습과제.ipynb',
 '1.지하철 이용승객 분석 EDA 프로젝트(1)완료.ipynb',
 'practice.ipynb',
 'rawfiles',
 'rawfiles.zip',
 'subway_raw.csv']

In [35]:
dirpath = 'rawfiles'
os.listdir(dirpath)

['rawfiles', '지하철노선위경도정보2.csv']

In [36]:
dirpath = 'C:/Users/limjy/Desktop/4학년 1학기/AI+X 선도인재양성/3주차_지하철 이용승객 EDA분석_숙제_20220914/실습/rawfiles/rawfiles'
files = os.listdir(dirpath)
files


['CARD_SUBWAY_MONTH_201901.csv',
 'CARD_SUBWAY_MONTH_201902.csv',
 'CARD_SUBWAY_MONTH_201903.csv',
 'CARD_SUBWAY_MONTH_201904.csv',
 'CARD_SUBWAY_MONTH_201905.csv',
 'CARD_SUBWAY_MONTH_201906.csv']

In [37]:
# 여러개의 파일을 병합하겠습니다.

# 빈 데이터프레임을 준비
raw = pd.DataFrame()

# 반복문 -> csv 파일 읽기 -> 데이터프레임에 추가하기
for file in files:
    #filepath = os.~ #(히든 파일이나 히든이 걸리면 제외시키고 하면됨.)
    #print(dirpath + file)
    temp = pd.read_csv(os.path.join(dirpath, file))  # 파일 경로 결합
    raw = pd.concat([raw, temp], axis=0)

raw.tail()


,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
17719,20190630,7호선,마들,6622.0,6116.0,20190703
17720,20190630,4호선,노원,14071.0,14702.0,20190703
17721,20190630,7호선,중계,9516.0,8981.0,20190703
17722,20190630,7호선,하계,11645.0,11121.0,20190703
17723,20190630,6호선,태릉입구,5730.0,5214.0,20190703


In [38]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107028 entries, 0 to 17723
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   사용일자    107028 non-null  int64  
 1   노선명     107028 non-null  object 
 2   역명      107028 non-null  object 
 3   승차총승객수  107028 non-null  float64
 4   하차총승객수  107028 non-null  float64
 5   등록일자    107028 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 5.7+ MB


In [39]:
raw = raw.reset_index(drop=True) #인덱스 초기화(여러개 합치는 경우 인덱스 초기화하는게 좋음=drop=True)

In [40]:
raw.head(7)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20190101,2호선,을지로4가,3862.0,3728.0,20190104
1,20190101,3호선,을지로3가,8104.0,7554.0,20190104
2,20190101,2호선,을지로입구,22478.0,21330.0,20190104
3,20190101,2호선,시청,8381.0,6049.0,20190104
4,20190101,6호선,동묘앞,8045.0,8504.0,20190104
5,20190101,경의선,청량리,15007.0,15397.0,20190104
6,20190101,1호선,제기동,10187.0,10178.0,20190104


# Q) 요일을 추가할수 있을까?

In [41]:
# 사용일자 컬럼 타입 확인 : int64

raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107028 entries, 0 to 107027
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   사용일자    107028 non-null  int64  
 1   노선명     107028 non-null  object 
 2   역명      107028 non-null  object 
 3   승차총승객수  107028 non-null  float64
 4   하차총승객수  107028 non-null  float64
 5   등록일자    107028 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 4.9+ MB


In [42]:
# '사용일자' 타입 int64 --> DateTime 타입으로 변경 : pandas.to_datetime 사용

raw['사용일자'] = pd.to_datetime(raw['사용일자'], format='%Y%m%d')
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107028 entries, 0 to 107027
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   사용일자    107028 non-null  datetime64[ns]
 1   노선명     107028 non-null  object        
 2   역명      107028 non-null  object        
 3   승차총승객수  107028 non-null  float64       
 4   하차총승객수  107028 non-null  float64       
 5   등록일자    107028 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 4.9+ MB


In [43]:
raw.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,2019-01-01,2호선,을지로4가,3862.0,3728.0,20190104
1,2019-01-01,3호선,을지로3가,8104.0,7554.0,20190104
2,2019-01-01,2호선,을지로입구,22478.0,21330.0,20190104
3,2019-01-01,2호선,시청,8381.0,6049.0,20190104
4,2019-01-01,6호선,동묘앞,8045.0,8504.0,20190104


### pandas.Series.dt
- 참고 사이트 : https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.dayofweek.html
  - pandas.Series.dt.date
  - pandas.Series.dt.time
  - pandas.Series.dt.year
  - pandas.Series.dt.month
  - pandas.Series.dt.day

In [44]:
#오류 확인할 때 Pandas 원전 파일인 위에 링크로 확인해야함.
#하지만 이미 답지를 만드셔서 일요일을 0으로 설정하고 진행하세요!!!

In [45]:
raw['사용일자'].dt.date.head(3)


0    2019-01-01
1    2019-01-01
2    2019-01-01
Name: 사용일자, dtype: object

In [46]:
raw['사용일자'].dt.time.head(3)

0    00:00:00
1    00:00:00
2    00:00:00
Name: 사용일자, dtype: object

In [47]:
raw['사용일자'].dt.year.head(3)

0    2019
1    2019
2    2019
Name: 사용일자, dtype: int32

In [48]:
raw['사용일자'].dt.month.head(3)

0    1
1    1
2    1
Name: 사용일자, dtype: int32

In [49]:
raw['사용일자'].dt.day.head(3)

0    1
1    1
2    1
Name: 사용일자, dtype: int32

In [50]:
# Monday=0, Sunday=6
raw['사용일자'].dt.dayofweek.head(3)

0    1
1    1
2    1
Name: 사용일자, dtype: int32

In [51]:
# 사용일자에 대해서 요일로 변환
raw['요일'] = raw['사용일자'].dt.dayofweek
raw.tail()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,요일
107023,2019-06-30,7호선,마들,6622.0,6116.0,20190703,6
107024,2019-06-30,4호선,노원,14071.0,14702.0,20190703,6
107025,2019-06-30,7호선,중계,9516.0,8981.0,20190703,6
107026,2019-06-30,7호선,하계,11645.0,11121.0,20190703,6
107027,2019-06-30,6호선,태릉입구,5730.0,5214.0,20190703,6


In [52]:
# 기존
conv = {
    0 : '일',
    1 : '월',
    2 : '화',
    3 : '수',
    4 : '목',
    5 : '금',
    6 : '토'
}

In [53]:
# 3칸 위에서 # Monday=0, Sunday=6라고 정의되어 있기 때문에, 이에 맞춰 변경함.
#conv = {
#    0 : '월',
#    1 : '화',
#    2 : '수',
#    3 : '목',
#    4 : '금',
#    5 : '토',
#    6 : '일'
#}

In [54]:
# 숫자 요일을 한글명 요일로 변경
raw['요일'] = raw['요일'].map(conv)


In [55]:
raw.sample(5)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,요일
63405,2019-04-18,1호선,가산디지털단지,20551.0,23846.0,20190421,수
975,2019-01-02,경의선,양평,7682.0,7763.0,20190105,화
53126,2019-03-31,7호선,광명사거리,17650.0,17112.0,20190403,토
92079,2019-06-05,5호선,고덕,12208.0,11599.0,20190608,화
16918,2019-01-29,분당선,이매,1225.0,1034.0,20190201,월


In [56]:
#사용일자에 따른 실제 요일 맞는지 확인 완료

# Q) '연월' 컬럼을 만들어 보자!

In [57]:
# pandas.Series.dt.strftime 활용
raw['사용일자'].dt.strftime('%Y-%m')

0         2019-01
1         2019-01
2         2019-01
3         2019-01
4         2019-01
           ...   
107023    2019-06
107024    2019-06
107025    2019-06
107026    2019-06
107027    2019-06
Name: 사용일자, Length: 107028, dtype: object

In [58]:
# 사용일자 컬럼에서 년월 값을 가져와 년월 컬럼 만들자

raw['연월'] = raw['사용일자'].dt.strftime('%Y-%m')
raw.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,요일,연월
0,2019-01-01,2호선,을지로4가,3862.0,3728.0,20190104,월,2019-01
1,2019-01-01,3호선,을지로3가,8104.0,7554.0,20190104,월,2019-01
2,2019-01-01,2호선,을지로입구,22478.0,21330.0,20190104,월,2019-01
3,2019-01-01,2호선,시청,8381.0,6049.0,20190104,월,2019-01
4,2019-01-01,6호선,동묘앞,8045.0,8504.0,20190104,월,2019-01


# Q) '월일' 컬럼을 만들어 보자!

In [59]:
# pandas.Series.dt.strftime 활용
raw['사용일자'].dt.strftime('%m-%d')

0         01-01
1         01-01
2         01-01
3         01-01
4         01-01
          ...  
107023    06-30
107024    06-30
107025    06-30
107026    06-30
107027    06-30
Name: 사용일자, Length: 107028, dtype: object

In [60]:
# 사용일자 컬럼에서 월일 값을 가져와 월일 컬럼 만들자

raw['월일'] = raw['사용일자'].dt.strftime('%m-%d')
raw.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,요일,연월,월일
0,2019-01-01,2호선,을지로4가,3862.0,3728.0,20190104,월,2019-01,01-01
1,2019-01-01,3호선,을지로3가,8104.0,7554.0,20190104,월,2019-01,01-01
2,2019-01-01,2호선,을지로입구,22478.0,21330.0,20190104,월,2019-01,01-01
3,2019-01-01,2호선,시청,8381.0,6049.0,20190104,월,2019-01,01-01
4,2019-01-01,6호선,동묘앞,8045.0,8504.0,20190104,월,2019-01,01-01


# Q) 승자와 하자하는 모든 승객수를 구해 볼까요?

In [61]:
# 승하자총승객수 컬럼 생성 : 승차총승객수 + 하차총승객수
raw['승하자총승객수'] = raw['승차총승객수'] + raw['하차총승객수']

In [62]:
# 요일을 사용일자 다음에 나오도록 컬럼순서를 변경하겠습니다.

# 현재 컬럼명 확인
print(raw.columns)

raw = raw[['사용일자', '요일', '노선명', '역명', '승차총승객수', '하차총승객수', '승하자총승객수', '연월', '월일', '등록일자' ]]
raw.head()


Index(['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자', '요일', '연월', '월일',
       '승하자총승객수'],
      dtype='object')


,사용일자,요일,노선명,역명,승차총승객수,하차총승객수,승하자총승객수,연월,월일,등록일자
0,2019-01-01,월,2호선,을지로4가,3862.0,3728.0,7590.0,2019-01,01-01,20190104
1,2019-01-01,월,3호선,을지로3가,8104.0,7554.0,15658.0,2019-01,01-01,20190104
2,2019-01-01,월,2호선,을지로입구,22478.0,21330.0,43808.0,2019-01,01-01,20190104
3,2019-01-01,월,2호선,시청,8381.0,6049.0,14430.0,2019-01,01-01,20190104
4,2019-01-01,월,6호선,동묘앞,8045.0,8504.0,16549.0,2019-01,01-01,20190104


# 3. 정리한 데이터 저장하기

In [63]:
# 작업한 결과를 data 폴더에 저장하겠습니다.

fpath = './subway_raw.csv'
raw.to_csv(fpath, index=False, encoding='utf-8-sig')
